In [1]:
# Import functions from helper python files.

import file_code, pairs_code, model_code
import importlib
from tqdm import tqdm
tqdm.pandas()

import pandas as pd

/home/sjain/anaconda3/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
/home/sjain/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/sjain/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
2025-10-17 18:46:40.340125: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to 

In [2]:
importlib.reload(file_code)
from file_code import *

In [3]:
files = get_files()

Reading raw data from text files. Generating files dataframe.


Read-in test files: 100%|███████████████████████████████████████| 2159/2159 [00:04<00:00, 451.30it/s]

Returning "files" df.


In [4]:
files_all = files

In [5]:
files_all

,filename,set,query,cases,text
0,059961,train,False,[],Indexed As: Canada (Minister of Citizenship an...
1,058794,train,False,[],<FRAGMENT_SUPPRESSED> (TD)\nMLB headnote and ...
2,057680,train,False,[],[1]\n: This is a judicial review application o...
3,075584,train,True,"[067672.txt, 019050.txt, 095238.txt, 047989.txt]",Introduction\n[1]\n[Translation]: It is well e...
4,085456,train,False,[],Summary:\nThe Immigration Appeal Division of t...
...,...,...,...,...,...
9284,076100,test,False,[],[1]\n: This application for judicial review co...
9285,035730,test,False,[],"Federal Court\nMactavish, J.\nFebruary 20, 200..."
9286,005193,test,False,[],[1]\n: This is an application for judicial rev...
9287,053188,test,True,[],"Federal Court\nFebruary 16, 2007.\nKingsley I...."


In [5]:
row_true = files_all[files_all['query'] == True].sample(n=1)
row_false = files_all[files_all['query'] == False].sample(n=1)

# Combine them
files = pd.concat([row_true, row_false])
files

,filename,set,query,cases,text
2512,079200,train,True,[089459.txt],"ATL Trucking Ltd. v. Port Authority (2015), 47..."
3316,063314,train,False,[],"[1]\nMosley, J.\n: Mr. Rith Kim is a 33-year-o..."


In [6]:
add_paragraphs(files)
files


Reading text from files. Extracting paragraphs based on regex pattern.


Extracting all paragraphs: 100%|█████████████████| 2/2 [00:00<00:00, 333.04it/s]

Updated "files" df has with "paragraphs".


,filename,set,query,cases,text,paragraphs
2512,079200,train,True,[089459.txt],"ATL Trucking Ltd. v. Port Authority (2015), 47...",[[1] : The applicants in both of these matters...
3316,063314,train,False,[],"[1]\nMosley, J.\n: Mr. Rith Kim is a 33-year-o...","[[1] Mosley, J. : Mr. Rith Kim is a 33-year-ol..."


In [7]:
get_paragraphs_formatted(files)
files

Getting formatted paragraphs of length < 250 words.


Getting formatted paragraphs: 100%|██████████████| 2/2 [00:00<00:00, 225.59it/s]

Added formatted paragraphs to "files" df in "paragraphs_formatted".


,filename,set,query,cases,text,paragraphs,paragraphs_formatted
2512,079200,train,True,[089459.txt],"ATL Trucking Ltd. v. Port Authority (2015), 47...",[[1] : The applicants in both of these matters...,[: The applicants in both of these matters are...
3316,063314,train,False,[],"[1]\nMosley, J.\n: Mr. Rith Kim is a 33-year-o...","[[1] Mosley, J. : Mr. Rith Kim is a 33-year-ol...","[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca..."


In [8]:
add_suppressed_sections(files)
files

Using regex and spacy (for long paragraphs) to extract and modify suppressed sections from paragraphs:


Extracting suppressed sections from paragraphs: 100%|█| 2/2 [00:00<00:00, 1084.0

Added suppressed sections to "suppressed_sections" field.


,filename,set,query,cases,text,paragraphs,paragraphs_formatted,suppressed_sections
2512,079200,train,True,[089459.txt],"ATL Trucking Ltd. v. Port Authority (2015), 47...",[[1] : The applicants in both of these matters...,[: The applicants in both of these matters are...,[[16] By granting or denying licenses to truck...
3316,063314,train,False,[],"[1]\nMosley, J.\n: Mr. Rith Kim is a 33-year-o...","[[1] Mosley, J. : Mr. Rith Kim is a 33-year-ol...","[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca...",[]


In [9]:
import re

def clean_supressed_section_list(section_list):
    """
    Cleans each string in supressed_section list individually:
    - Removes citation numbers like [1], [2], etc.
    - Removes uppercase 'REFERENCE' and 'TARGETCASE' (with or without <>)
    - Removes 'supra'
    - Removes empty parentheses and extra punctuation
    - Strips extra spaces
    - Keeps only unique sentences per section
    """
    if not section_list:
        return []
    
    if not isinstance(section_list, list):
        section_list = [section_list]
    
    cleaned_list = []
    for text in section_list:
        # Remove citation numbers like [1], [23], etc.
        text = re.sub(r"\[\d+\]", "", text)
        # Remove uppercase REFERENCE/TARGETCASE (with or without < >)
        text = re.sub(r"<*REFERENCE>*|<*TARGETCASE>*", "", text)
        # Remove 'supra'
        text = re.sub(r"\bsupra\b", "", text, flags=re.IGNORECASE)
        # Remove empty parentheses
        text = re.sub(r"\(\s*\)", "", text)
        # Remove multiple punctuation marks and extra spaces
        text = re.sub(r"[;,\s]{2,}", " ", text)
        text = re.sub(r"\s+", " ", text)
        # Strip leading/trailing spaces
        text = text.strip()
        if text:  # only keep non-empty strings
            cleaned_list.append(text)
    
    # Remove duplicates within this section while preserving order
    seen = set()
    unique_list = []
    for item in cleaned_list:
        if item not in seen:
            seen.add(item)
            unique_list.append(item)
    
    return unique_list

    
files["propositions"] = files["suppressed_sections"].apply(clean_supressed_section_list)
files

,filename,set,query,cases,text,paragraphs,paragraphs_formatted,suppressed_sections,propositions
2512,079200,train,True,[089459.txt],"ATL Trucking Ltd. v. Port Authority (2015), 47...",[[1] : The applicants in both of these matters...,[: The applicants in both of these matters are...,[[16] By granting or denying licenses to truck...,[By granting or denying licenses to trucking c...
3316,063314,train,False,[],"[1]\nMosley, J.\n: Mr. Rith Kim is a 33-year-o...","[[1] Mosley, J. : Mr. Rith Kim is a 33-year-ol...","[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca...",[],[]


In [10]:
files["propositions_en"] = files["propositions"]
files

,filename,set,query,cases,text,paragraphs,paragraphs_formatted,suppressed_sections,propositions,propositions_en
2512,079200,train,True,[089459.txt],"ATL Trucking Ltd. v. Port Authority (2015), 47...",[[1] : The applicants in both of these matters...,[: The applicants in both of these matters are...,[[16] By granting or denying licenses to truck...,[By granting or denying licenses to trucking c...,[By granting or denying licenses to trucking c...
3316,063314,train,False,[],"[1]\nMosley, J.\n: Mr. Rith Kim is a 33-year-o...","[[1] Mosley, J. : Mr. Rith Kim is a 33-year-ol...","[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca...",[],[],[]


In [11]:
add_sentences(files)

Using spacy to extract sentences of char length > 25 from paragraphs:


Extracting sentences from paragraphs: 100%|███████| 2/2 [00:02<00:00,  1.04s/it]

Added lists of sentences to "sentences".


In [12]:
for s in files['sentences'].iloc[:2]:
    print(len(s))   # how many sentences per row?

57
196


In [13]:
files

,filename,set,query,cases,text,paragraphs,paragraphs_formatted,suppressed_sections,propositions,propositions_en,sentences
2512,079200,train,True,[089459.txt],"ATL Trucking Ltd. v. Port Authority (2015), 47...",[[1] : The applicants in both of these matters...,[: The applicants in both of these matters are...,[[16] By granting or denying licenses to truck...,[By granting or denying licenses to trucking c...,[By granting or denying licenses to trucking c...,[: The applicants in both of these matters are...
3316,063314,train,False,[],"[1]\nMosley, J.\n: Mr. Rith Kim is a 33-year-o...","[[1] Mosley, J. : Mr. Rith Kim is a 33-year-ol...","[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca...",[],[],[],"[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca..."


In [14]:
files["sentences_en"] = files["sentences"]
files

,filename,set,query,cases,text,paragraphs,paragraphs_formatted,suppressed_sections,propositions,propositions_en,sentences,sentences_en
2512,079200,train,True,[089459.txt],"ATL Trucking Ltd. v. Port Authority (2015), 47...",[[1] : The applicants in both of these matters...,[: The applicants in both of these matters are...,[[16] By granting or denying licenses to truck...,[By granting or denying licenses to trucking c...,[By granting or denying licenses to trucking c...,[: The applicants in both of these matters are...,[: The applicants in both of these matters are...
3316,063314,train,False,[],"[1]\nMosley, J.\n: Mr. Rith Kim is a 33-year-o...","[[1] Mosley, J. : Mr. Rith Kim is a 33-year-ol...","[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca...",[],[],[],"[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca...","[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca..."


In [15]:
add_quotes(files)
add_entities(files)
add_strings_sets(files)
files

Using regex to extract quotations from suppressed sections:


Extracting quotes from suppressed sections:: 100%|█| 2/2 [00:00<00:00, 376.49it/

Added quotes to the "quotes" field for query cases.
Using spacy to extract noun entities, from English sentences:



Extracting entities from english sentences: 100%|█| 2/2 [00:02<00:00,  1.05s/it]

Added entity strings to "entity_string" and entities as sets to "entity_set".
Extracting case word strings (for tfidf) and sets (for case jaccard):



Extracting case string and sets from sentences_en:: 100%|█| 2/2 [00:00<00:00, 18

Added case strings and sets.


,filename,set,query,cases,text,paragraphs,paragraphs_formatted,suppressed_sections,propositions,propositions_en,sentences,sentences_en,quotes,entity_string,entity_set,sentences_en_string,sentences_en_set
2512,079200,train,True,[089459.txt],"ATL Trucking Ltd. v. Port Authority (2015), 47...",[[1] : The applicants in both of these matters...,[: The applicants in both of these matters are...,[[16] By granting or denying licenses to truck...,[By granting or denying licenses to trucking c...,[By granting or denying licenses to trucking c...,[: The applicants in both of these matters are...,[: The applicants in both of these matters are...,[],vfpa tls vfpa federalcourt vfpa federalcourtsa...,"{canadamarineact, supremecourtcanada, canada, ...",applic matter truck compani appli licens new t...,"{enterpris, background, paragraph, describ, re..."
3316,063314,train,False,[],"[1]\nMosley, J.\n: Mr. Rith Kim is a 33-year-o...","[[1] Mosley, J. : Mr. Rith Kim is a 33-year-ol...","[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca...",[],[],[],"[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca...","[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca...",[],cambodian canada canada unitednations indochi...,"{, supremecourt, barry, federalcourtappeal, ca...",mosley j mr rith kim cambodian nation arriv ca...,"{explicitli, express, initi, predetermin, open..."


In [16]:
add_set_lists(files)
add_judge_name(files)
add_year(files)

Extracting set lists:


Extracting set list from propositions_en:: 100%|█| 2/2 [00:00<00:00, 693.22it/s]

Added set lists.
Using regex to extract judge surname from first paragraphs:



Extracting judge surname from paragraphs: 100%|█| 2/2 [00:00<00:00, 1627.28it/s]

Added judge name to "judge" field.
Using string search to find year:



Extracting most recent year from file text: 100%|█| 2/2 [00:00<00:00, 382.10it/s

Added year to files.


In [17]:
files

,filename,set,query,cases,text,paragraphs,paragraphs_formatted,suppressed_sections,propositions,propositions_en,...,quotes,entity_string,entity_set,sentences_en_string,sentences_en_set,sentences_en_set_list,paragraphs_formatted_en_set_list,propositions_en_set_list,judge,year
2512,079200,train,True,[089459.txt],"ATL Trucking Ltd. v. Port Authority (2015), 47...",[[1] : The applicants in both of these matters...,[: The applicants in both of these matters are...,[[16] By granting or denying licenses to truck...,[By granting or denying licenses to trucking c...,[By granting or denying licenses to trucking c...,...,[],vfpa tls vfpa federalcourt vfpa federalcourtsa...,"{canadamarineact, supremecourtcanada, canada, ...",applic matter truck compani appli licens new t...,"{enterpris, background, paragraph, describ, re...","[{applic, marin, matter, implement, oper, new,...","[{applic, marin, obtain, neg, matter, implemen...","[{applic, natur, support, canada, dealt, john,...",None,2015
3316,063314,train,False,[],"[1]\nMosley, J.\n: Mr. Rith Kim is a 33-year-o...","[[1] Mosley, J. : Mr. Rith Kim is a 33-year-ol...","[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca...",[],[],[],...,[],cambodian canada canada unitednations indochi...,"{, supremecourt, barry, federalcourtappeal, ca...",mosley j mr rith kim cambodian nation arriv ca...,"{explicitli, express, initi, predetermin, open...","[{nation, mosley, rith, mr, cambodian, canada,...","[{mandat, resettl, southeast, canada, have, fa...",[],Mosley,2003


In [19]:
get_embeddings(files)
files.to_csv('embeddings_new_2.csv', index=False)
files

Getting embeddings from sentences, paragraphs and propositions.


/home/sjain/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Getting embeddings for sentences_en: 100%|████████| 2/2 [00:58<00:00, 29.14s/it]
Getting embeddings for paragraphs formatted: 100%|█| 2/2 [00:40<00:00, 20.29s/it
Getting embeddings for propositions_en: 100%|█████| 2/2 [00:00<00:00,  5.18it/s]

Added embeddings.


,filename,set,query,cases,text,paragraphs,paragraphs_formatted,suppressed_sections,propositions,propositions_en,...,sentences_en_string,sentences_en_set,sentences_en_set_list,paragraphs_formatted_en_set_list,propositions_en_set_list,judge,year,embeddings_sentences_en,embeddings_paragraphs_formatted,embeddings_propositions_en
2512,079200,train,True,[089459.txt],"ATL Trucking Ltd. v. Port Authority (2015), 47...",[[1] : The applicants in both of these matters...,[: The applicants in both of these matters are...,[[16] By granting or denying licenses to truck...,[By granting or denying licenses to trucking c...,[By granting or denying licenses to trucking c...,...,applic matter truck compani appli licens new t...,"{enterpris, background, paragraph, describ, re...","[{applic, marin, matter, implement, oper, new,...","[{applic, marin, obtain, neg, matter, implemen...","[{applic, natur, support, canada, dealt, john,...",None,2015,"[[-0.050765198, 0.022895811, -0.089684404, 0.0...","[[-0.05379617, -0.0071261334, -0.039556436, 0....","[[-0.0040527657, 0.026140483, -0.014018228, 0...."
3316,063314,train,False,[],"[1]\nMosley, J.\n: Mr. Rith Kim is a 33-year-o...","[[1] Mosley, J. : Mr. Rith Kim is a 33-year-ol...","[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca...",[],[],[],...,mosley j mr rith kim cambodian nation arriv ca...,"{explicitli, express, initi, predetermin, open...","[{nation, mosley, rith, mr, cambodian, canada,...","[{mandat, resettl, southeast, canada, have, fa...",[],Mosley,2003,"[[0.117795825, 0.083978675, 0.069928735, 0.032...","[[0.08513829, 0.09310895, 0.04920223, -0.04336...",[]


In [6]:
# files.to_pickle("embedding_pkl.pkl")
files = pd.read_pickle("embedding_pkl.pkl")
files

,filename,set,query,cases,text,paragraphs,paragraphs_formatted,suppressed_sections,propositions,propositions_en,...,sentences_en_string,sentences_en_set,sentences_en_set_list,paragraphs_formatted_en_set_list,propositions_en_set_list,judge,year,embeddings_sentences_en,embeddings_paragraphs_formatted,embeddings_propositions_en
2512,079200,train,True,[089459.txt],"ATL Trucking Ltd. v. Port Authority (2015), 47...",[[1] : The applicants in both of these matters...,[: The applicants in both of these matters are...,[[16] By granting or denying licenses to truck...,[By granting or denying licenses to trucking c...,[By granting or denying licenses to trucking c...,...,applic matter truck compani appli licens new t...,"{ought, purpos, transport, refer, dismiss, end...","[{tl, licens, oper, matter, marin, creat, impl...","[{judici, tl, licens, decis, oper, marin, obta...","[{privat, tour, licens, dealt, boat, suprem, t...",None,2015,"[[-0.050765198, 0.022895811, -0.089684404, 0.0...","[[-0.05379617, -0.0071261334, -0.039556436, 0....","[[-0.0040527657, 0.026140483, -0.014018228, 0...."
3316,063314,train,False,[],"[1]\nMosley, J.\n: Mr. Rith Kim is a 33-year-o...","[[1] Mosley, J. : Mr. Rith Kim is a 33-year-ol...","[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca...",[],[],[],...,mosley j mr rith kim cambodian nation arriv ca...,"{d, revok, dismiss, religion, presumpt, scr, o...","[{kim, canada, mr, mosley, nation, age, rith, ...","[{nation, boat, resettl, class, high, resid, a...",[],Mosley,2003,"[[0.117795825, 0.083978675, 0.069928735, 0.032...","[[0.08513829, 0.09310895, 0.04920223, -0.04336...",[]


In [8]:
importlib.reload(pairs_code)
from pairs_code import (add_bins, get_prop_max_cos_sim_sents, get_prop_max_cos_sim_paras,
                        get_prop_max_jaccard_sents, get_prop_max_jaccard_paras, get_prop_max_overlap_sents, get_prop_max_overlap_paras, add_max_overall,
                        get_case_jaccard_sims, check_same_case, get_case_tfidf_scores, get_num_quotes, binarize_quotes, check_years, add_judge_checks)

In [12]:
import pandas as pd

# Load both CSVs
df1 = pd.read_csv("bm25_results_check.csv")
df2 = pd.read_csv("bm25_results_check_part2.csv")


In [13]:
merged = pd.concat([df1, df2], ignore_index=True)
merged

,query_filename,paragraph,candidates,scores
0,75584,Shore J. [Translation]: It is well established...,"['012703.txt', '043794.txt', '086440.txt', '08...","[393.3033183557051, 373.1294400328441, 370.992..."
1,75584,The question of whether a relationship is genu...,"['072821.txt', '051020.txt', '036471.txt', '07...","[282.0421465112925, 279.2595987917241, 277.066..."
2,75584,The case law of this Court has clearly establi...,"['012703.txt', '043794.txt', '067440.txt', '09...","[364.3798724873776, 352.9402874066394, 346.098..."
3,75584,As the respondent noted at paragraph 4 of his ...,"['045084.txt', '088086.txt', '020568.txt', '02...","[183.9995696957007, 182.05958732569562, 182.05..."
4,75584,Overall the Court finds the impugned decision ...,"['008240.txt', '042250.txt', '071972.txt', '00...","[210.06461728717056, 209.09726934527703, 208.3..."
...,...,...,...,...
27906,53188,The respondent submitted that by the applicant...,"['017170.txt', '021417.txt', '037569.txt', '07...","[642.3685097948018, 506.9399525502303, 497.188..."
27907,53188,The respondent submitted that it was open to t...,"['017170.txt', '026200.txt', '020662.txt', '06...","[214.69374155133525, 185.8904816662488, 183.66..."
27908,53188,The respondent submitted that it was the appli...,"['017170.txt', '046028.txt', '039471.txt', '00...","[545.9547400992672, 458.9765402690308, 448.671..."
27909,53188,The respondent submitted that the Board may di...,"['017170.txt', '026200.txt', '069962.txt', '02...","[320.51164635316223, 284.2654879455327, 283.78..."


In [15]:
import pandas as pd
import ast

def revert_dataframe_columns(df):
    """
    Attempts to convert stringified lists/dicts in all columns
    back to their original Python objects.
    """
    for col in df.columns:
        def try_eval(x):
            if isinstance(x, str):
                try:
                    return ast.literal_eval(x)
                except (ValueError, SyntaxError):
                    return x  # keep as string if it can't be evaluated
            return x
        df[col] = df[col].apply(try_eval)
    return df

revert_dataframe_columns(merged)
merged

<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal


,query_filename,paragraph,candidates,scores
0,75584,Shore J. [Translation]: It is well established...,"[012703.txt, 043794.txt, 086440.txt, 082714.tx...","[393.3033183557051, 373.1294400328441, 370.992..."
1,75584,The question of whether a relationship is genu...,"[072821.txt, 051020.txt, 036471.txt, 074145.tx...","[282.0421465112925, 279.2595987917241, 277.066..."
2,75584,The case law of this Court has clearly establi...,"[012703.txt, 043794.txt, 067440.txt, 099980.tx...","[364.3798724873776, 352.9402874066394, 346.098..."
3,75584,As the respondent noted at paragraph 4 of his ...,"[045084.txt, 088086.txt, 020568.txt, 027284.tx...","[183.9995696957007, 182.05958732569562, 182.05..."
4,75584,Overall the Court finds the impugned decision ...,"[008240.txt, 042250.txt, 071972.txt, 001540.tx...","[210.06461728717056, 209.09726934527703, 208.3..."
...,...,...,...,...
27906,53188,The respondent submitted that by the applicant...,"[017170.txt, 021417.txt, 037569.txt, 075160.tx...","[642.3685097948018, 506.9399525502303, 497.188..."
27907,53188,The respondent submitted that it was open to t...,"[017170.txt, 026200.txt, 020662.txt, 069189.tx...","[214.69374155133525, 185.8904816662488, 183.66..."
27908,53188,The respondent submitted that it was the appli...,"[017170.txt, 046028.txt, 039471.txt, 001630.tx...","[545.9547400992672, 458.9765402690308, 448.671..."
27909,53188,The respondent submitted that the Board may di...,"[017170.txt, 026200.txt, 069962.txt, 028335.tx...","[320.51164635316223, 284.2654879455327, 283.78..."


In [16]:
import pandas as pd

# Step 2: group by filename and union candidate lists
new_df = (
    merged.groupby("query_filename", as_index=False)
          .agg({
              "candidates": lambda x: list(set(sum(x, [])))  # union of all lists
          })
)


In [17]:
new_df

,query_filename,candidates
0,2,"[054392.txt, 054431.txt, 072955.txt, 093012.tx..."
1,31,"[070627.txt, 018972.txt, 047149.txt, 040209.tx..."
2,68,"[032157.txt, 061239.txt, 070773.txt, 057165.tx..."
3,85,"[008365.txt, 088344.txt, 021073.txt, 018972.tx..."
4,92,"[061913.txt, 079616.txt, 072955.txt, 038641.tx..."
...,...,...
2071,99856,"[079616.txt, 018972.txt, 076852.txt, 032157.tx..."
2072,99874,"[025707.txt, 037135.txt, 066946.txt, 057846.tx..."
2073,99892,"[001100.txt, 044411.txt, 061913.txt, 036037.tx..."
2074,99898,"[054392.txt, 031558.txt, 051636.txt, 010333.tx..."


In [18]:
print(type(merged["candidates"].iloc[0]))


<class 'list'>


In [19]:
new_df["candidate_len"] = new_df["candidates"].apply(len)


In [20]:
new_df

,query_filename,candidates,candidate_len
0,2,"[054392.txt, 054431.txt, 072955.txt, 093012.tx...",177
1,31,"[070627.txt, 018972.txt, 047149.txt, 040209.tx...",407
2,68,"[032157.txt, 061239.txt, 070773.txt, 057165.tx...",315
3,85,"[008365.txt, 088344.txt, 021073.txt, 018972.tx...",212
4,92,"[061913.txt, 079616.txt, 072955.txt, 038641.tx...",327
...,...,...,...
2071,99856,"[079616.txt, 018972.txt, 076852.txt, 032157.tx...",492
2072,99874,"[025707.txt, 037135.txt, 066946.txt, 057846.tx...",192
2073,99892,"[001100.txt, 044411.txt, 061913.txt, 036037.tx...",451
2074,99898,"[054392.txt, 031558.txt, 051636.txt, 010333.tx...",289


In [21]:
new_df.to_pickle("top_50_union.pkl")

In [22]:
new_df['candidates'] = new_df['candidates'].apply(lambda lst: [x.replace('.txt', '') for x in lst])

In [23]:
new_df

,query_filename,candidates,candidate_len
0,2,"[054392, 054431, 072955, 093012, 044060, 00937...",177
1,31,"[070627, 018972, 047149, 040209, 098337, 00256...",407
2,68,"[032157, 061239, 070773, 057165, 010575, 08100...",315
3,85,"[008365, 088344, 021073, 018972, 016280, 06935...",212
4,92,"[061913, 079616, 072955, 038641, 098227, 01538...",327
...,...,...,...
2071,99856,"[079616, 018972, 076852, 032157, 038641, 08239...",492
2072,99874,"[025707, 037135, 066946, 057846, 098047, 08856...",192
2073,99892,"[001100, 044411, 061913, 036037, 018972, 08894...",451
2074,99898,"[054392, 031558, 051636, 010333, 031230, 00224...",289


In [24]:
files_all

,filename,set,query,cases,text
0,059961,train,False,[],Indexed As: Canada (Minister of Citizenship an...
1,058794,train,False,[],<FRAGMENT_SUPPRESSED> (TD)\nMLB headnote and ...
2,057680,train,False,[],[1]\n: This is a judicial review application o...
3,075584,train,True,"[067672.txt, 019050.txt, 095238.txt, 047989.txt]",Introduction\n[1]\n[Translation]: It is well e...
4,085456,train,False,[],Summary:\nThe Immigration Appeal Division of t...
...,...,...,...,...,...
9284,076100,test,False,[],[1]\n: This application for judicial review co...
9285,035730,test,False,[],"Federal Court\nMactavish, J.\nFebruary 20, 200..."
9286,005193,test,False,[],[1]\n: This is an application for judicial rev...
9287,053188,test,True,[],"Federal Court\nFebruary 16, 2007.\nKingsley I...."


In [26]:
files_all['cases'] = files_all['cases'].apply(lambda lst: [x.replace('.txt', '') for x in lst])
files_all

,filename,set,query,cases,text
0,059961,train,False,[],Indexed As: Canada (Minister of Citizenship an...
1,058794,train,False,[],<FRAGMENT_SUPPRESSED> (TD)\nMLB headnote and ...
2,057680,train,False,[],[1]\n: This is a judicial review application o...
3,075584,train,True,"[067672, 019050, 095238, 047989]",Introduction\n[1]\n[Translation]: It is well e...
4,085456,train,False,[],Summary:\nThe Immigration Appeal Division of t...
...,...,...,...,...,...
9284,076100,test,False,[],[1]\n: This application for judicial review co...
9285,035730,test,False,[],"Federal Court\nMactavish, J.\nFebruary 20, 200..."
9286,005193,test,False,[],[1]\n: This is an application for judicial rev...
9287,053188,test,True,[],"Federal Court\nFebruary 16, 2007.\nKingsley I...."


In [27]:
matching_rows = files_all.loc[files_all['filename'] == query_filename, 'set']

if len(matching_rows) == 0:
    print(f"No match found for {query_filename}")
    query_set = 'unknown'  # or skip this row
else:
    query_set = matching_rows.values[0]


NameError: name 'query_filename' is not defined

In [28]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def get_pairs_with_candidates(files_all, new_df):
    """
    Generate query-target pairs using candidates from new_df.
    Only processes queries where files_all['query'] == True.
    Handles .txt stripping, padding, and missing filenames safely.
    """

    tqdm.pandas(desc="Getting matches")
    all_pairs = []

    # Filter only true queries
    files_all_queries = files_all[files_all['query'] == True]

    # Convert 'cases' to dict for easy lookup
    cases_dict = files_all_queries.set_index('filename')['cases'].to_dict()

    for idx, row in new_df.iterrows():
        # Normalize query filename
        query_filename_str = str(row['query_filename']).zfill(6)

        # Safe lookup for set
        matching_rows = files_all_queries.loc[
            files_all_queries['filename'] == query_filename_str, 'set'
        ]
        if len(matching_rows) == 0:
            # Skip if query not found in files_all
            continue
        query_set = matching_rows.values[0]

        # Clean candidates: remove .txt and pad
        candidates_clean = [str(c).rstrip('.txt').zfill(6) for c in row['candidates']]

        for target_clean in candidates_clean:
            # Skip if query == target
            if query_filename_str == target_clean:
                continue

            # Determine match
            match = np.nan
            if query_filename_str in cases_dict:
                match = 1 if target_clean in [str(x).zfill(6) for x in cases_dict[query_filename_str]] else 0
                if query_set == 'test':
                    match = np.nan

            all_pairs.append({
                'query': query_filename_str,
                'target': target_clean,
                'tuple': (query_filename_str, target_clean),
                'set': query_set,
                'match': match
            })

    pairs_df = pd.DataFrame(all_pairs)
    return pairs_df

# Usage
pairs = get_pairs_with_candidates(files_all, new_df)
pairs.head()


,query,target,tuple,set,match
0,000002,054392,"(000002, 054392)",train,0.0
1,000002,054431,"(000002, 054431)",train,0.0
2,000002,072955,"(000002, 072955)",train,0.0
3,000002,093012,"(000002, 093012)",train,0.0
4,000002,044060,"(000002, 044060)",train,0.0


In [29]:
pairs

,query,target,tuple,set,match
0,000002,054392,"(000002, 054392)",train,0.0
1,000002,054431,"(000002, 054431)",train,0.0
2,000002,072955,"(000002, 072955)",train,0.0
3,000002,093012,"(000002, 093012)",train,0.0
4,000002,044060,"(000002, 044060)",train,0.0
...,...,...,...,...,...
624010,099982,032076,"(099982, 032076)",test,NaN
624011,099982,087440,"(099982, 087440)",test,NaN
624012,099982,075123,"(099982, 075123)",test,NaN
624013,099982,071484,"(099982, 071484)",test,NaN


In [30]:
# Filter rows where match == 1
matched_rows = pairs[pairs['match'] == 1]

# Print them
print(matched_rows)


         query  target             tuple    set  match
196     000031  091216  (000031, 091216)  train    1.0
356     000031  082701  (000031, 082701)  train    1.0
435     000031  070067  (000031, 070067)  train    1.0
447     000031  013059  (000031, 013059)  train    1.0
492     000031  074681  (000031, 074681)  train    1.0
...        ...     ...               ...    ...    ...
622726  099856  079743  (099856, 079743)  train    1.0
623003  099892  097422  (099892, 097422)  train    1.0
623107  099892  094309  (099892, 094309)  train    1.0
623256  099892  035289  (099892, 035289)  train    1.0
623447  099898  051636  (099898, 051636)  train    1.0

[4661 rows x 5 columns]


In [31]:
pairs.to_pickle("pairs_after_filter.pkl")
# save kar liya

In [32]:
pairs

,query,target,tuple,set,match
0,000002,054392,"(000002, 054392)",train,0.0
1,000002,054431,"(000002, 054431)",train,0.0
2,000002,072955,"(000002, 072955)",train,0.0
3,000002,093012,"(000002, 093012)",train,0.0
4,000002,044060,"(000002, 044060)",train,0.0
...,...,...,...,...,...
624010,099982,032076,"(099982, 032076)",test,NaN
624011,099982,087440,"(099982, 087440)",test,NaN
624012,099982,075123,"(099982, 075123)",test,NaN
624013,099982,071484,"(099982, 071484)",test,NaN


In [34]:
from pairs_code import get_pairs
pair_new = get_pairs(files_all)

Generating pairs dataframe from files.


Getting matches: 100%|██████████████████████████████████| 9981185/9981185 [01:44<00:00, 95372.52it/s]


In [35]:
pair_new
# apna code shi hai

,query,target,tuple,set,match
0,000002,000028,"(000002, 000028)",train,0.0
1,000002,000054,"(000002, 000054)",train,0.0
2,000002,000084,"(000002, 000084)",train,0.0
3,000002,000094,"(000002, 000094)",train,0.0
4,000002,000109,"(000002, 000109)",train,0.0
...,...,...,...,...,...
9981180,099982,099505,"(099982, 099505)",test,NaN
9981181,099982,099507,"(099982, 099507)",test,NaN
9981182,099982,099523,"(099982, 099523)",test,NaN
9981183,099982,099544,"(099982, 099544)",test,NaN


In [36]:
twofile = pd.read_pickle("embedding_pkl.pkl")
twofile

,filename,set,query,cases,text,paragraphs,paragraphs_formatted,suppressed_sections,propositions,propositions_en,...,sentences_en_string,sentences_en_set,sentences_en_set_list,paragraphs_formatted_en_set_list,propositions_en_set_list,judge,year,embeddings_sentences_en,embeddings_paragraphs_formatted,embeddings_propositions_en
2512,079200,train,True,[089459.txt],"ATL Trucking Ltd. v. Port Authority (2015), 47...",[[1] : The applicants in both of these matters...,[: The applicants in both of these matters are...,[[16] By granting or denying licenses to truck...,[By granting or denying licenses to trucking c...,[By granting or denying licenses to trucking c...,...,applic matter truck compani appli licens new t...,"{ought, purpos, transport, refer, dismiss, end...","[{tl, licens, oper, matter, marin, creat, impl...","[{judici, tl, licens, decis, oper, marin, obta...","[{privat, tour, licens, dealt, boat, suprem, t...",None,2015,"[[-0.050765198, 0.022895811, -0.089684404, 0.0...","[[-0.05379617, -0.0071261334, -0.039556436, 0....","[[-0.0040527657, 0.026140483, -0.014018228, 0...."
3316,063314,train,False,[],"[1]\nMosley, J.\n: Mr. Rith Kim is a 33-year-o...","[[1] Mosley, J. : Mr. Rith Kim is a 33-year-ol...","[Mosley, J. : Mr. Rith Kim is a 33-year-old Ca...",[],[],[],...,mosley j mr rith kim cambodian nation arriv ca...,"{d, revok, dismiss, religion, presumpt, scr, o...","[{kim, canada, mr, mosley, nation, age, rith, ...","[{nation, boat, resettl, class, high, resid, a...",[],Mosley,2003,"[[0.117795825, 0.083978675, 0.069928735, 0.032...","[[0.08513829, 0.09310895, 0.04920223, -0.04336...",[]


In [37]:
get_prop_max_cos_sim_sents(twofile, pairs)

Getting max cos similarity between query propositions and target sentences_en.


Getting max cosine similarity: 100%|█████████████████████| 624015/624015 [00:04<00:00, 152580.02it/s]

Maximum proposition cosine similarity (with sentences_en) added to pairs.


In [47]:
pairs

,query,target,tuple,set,match,prop_max_cos_sim_sents,prop_max_cos_sim_paras
0,000002,054392,"(000002, 054392)",train,0.0,0,0
1,000002,054431,"(000002, 054431)",train,0.0,0,0
2,000002,072955,"(000002, 072955)",train,0.0,0,0
3,000002,093012,"(000002, 093012)",train,0.0,0,0
4,000002,044060,"(000002, 044060)",train,0.0,0,0
...,...,...,...,...,...,...,...
624010,099982,032076,"(099982, 032076)",test,NaN,0,0
624011,099982,087440,"(099982, 087440)",test,NaN,0,0
624012,099982,075123,"(099982, 075123)",test,NaN,0,0
624013,099982,071484,"(099982, 071484)",test,NaN,0,0


In [48]:
non_zero_rows = pairs[pairs['prop_max_cos_sim_sents'] != 0]
non_zero_rows = pairs[pairs['prop_max_cos_sim_paras'] != 0]
non_zero_rows

,query,target,tuple,set,match,prop_max_cos_sim_sents,prop_max_cos_sim_paras


In [46]:
get_prop_max_cos_sim_paras(twofile, pairs)

Getting max cos similarity between query propositions and target paragraphs_formatted.


Getting max cosine similarity: 100%|█████████████████████| 624015/624015 [00:04<00:00, 151061.30it/s]

Maximum proposition cosine similarity (with paragraphs_formatted) added to pairs.


In [49]:
get_prop_max_jaccard_sents(twofile,pairs)
get_prop_max_jaccard_paras(twofile,pairs)
pairs

Getting max jaccard similarity between sentences_en_sets and propositions_en_sets.


Getting max jaccard proposition-sentences:   0%|               | 1/624015 [00:00<37:09:44,  4.66it/s]


KeyError: '000002'

In [41]:


get_prop_max_overlap_sents(twofile,pairs)
get_prop_max_overlap_paras(files,pairs)
add_max_overall(pairs,files)
get_case_jaccard_sims(files,pairs)
check_same_case(pairs)
get_case_tfidf_scores(files,pairs)
get_num_quotes(files,pairs)
binarize_quotes(pairs)
check_years(files,pairs)
add_judge_checks(files,pairs)
add_bins(files, pairs)

Getting max cos similarity between query propositions and target paragraphs_formatted.


Getting max cosine similarity: 100%|█████████████████████| 624015/624015 [00:04<00:00, 155797.81it/s]

Maximum proposition cosine similarity (with paragraphs_formatted) added to pairs.
Getting max overlap ratio between sentences_en_sets and propositions_en_sets.



Getting max overlap proposition-sentences:   0%|               | 1/624015 [00:00<32:40:45,  5.30it/s]


KeyError: '000002'